### add column 

In [103]:
def add_cols(jpath, fpath, cols):
    jpath = '../data/pd_gy/all.json'
    fpath = 'filter0.csv'

    add_cols = ['age','gender']

    import json 
    with open(jpath,'r') as f:
        dj = json.load(f)

    import pandas as pd
    op = pd.read_csv(fpath)

    op = op.rename(columns = {'성별':'gender','나이':'age','KEY':'key'})
    op['key'] = op.key.astype('str')

    l = []
    for i,val in enumerate(dj):
        key  = val['key']
        item = op[op.key==key]
        for col in add_cols:
            col_value = item[col].values[0]
            dj[i][col] = col_value
    return dj

In [124]:
jpath = '../data/pd_gy/all.json'
fpath = 'filter0.csv'

import json 
with open(jpath,'r') as f:
    dj = json.load(f)

import pandas as pd
op = pd.read_csv(fpath)

op = op.rename(columns = {'성별':'gender','나이':'age','KEY':'key'})
op['key'] = op.key.astype('str')

l = []
for i,val in enumerate(dj):
    key  = val['key']
    item = op[op.key==key]
    age = int(item['age'].values[0])
    gender = str(item['gender'].values[0])
    dj[i]['age'], dj[i]['gender'] = age, gender.lower()

In [126]:
import json
with open('../data/pd_gy/all_1.json','w') as f:
    json.dump(dj,f)

### 3 hold out 

In [1]:
jpath = '../data/pd_gy/all_1.json'

import json
with open(jpath, 'r') as f:
    jf = json.load(f)

In [2]:
from utils import num2class

In [3]:
for i,val in enumerate(jf):
    jf[i]['nrs_2'] = num2class(val['nrs'])

In [15]:
import pandas as pd

from sklearn.model_selection import train_test_split

# train:val:test = 3:1:1
nrs2_0 = pd.DataFrame(jf)['nrs_2']
xtr, xte= train_test_split(jf, test_size=0.2, shuffle=True, stratify=nrs2_0, random_state=34)
nrs2_1 = pd.DataFrame(xtr)['nrs_2']
xtr, xval= train_test_split(xtr, test_size=0.25, shuffle=True, stratify=nrs2_1, random_state=34)

In [ ]:
with open('../data/pd_gy/train.json','w') as f:
    json.dump(xtr,f)
with open('../data/pd_gy/val.json','w') as f:
    json.dump(xval,f)
with open('../data/pd_gy/test.json','w') as f:
    json.dump(xte,f)

### PPG 

### PPG + [name, age] 